# Classes & Databases

In [736]:
# fixing autocomplete
%config Completer.use_jedi = False
# better quality plots in notebook
%config InlineBackend.figure_format='retina'
# cell timings
%load_ext autotime
import pandas as pd
import sympy as sp
import pickle
import json
from matplotlib import pyplot as plt
import numpy as np
%matplotlib widget

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


For its approximation of the optical properties of transition metal ions in crystal hosts, **qdef** takes information from the following sources:

* From NIST it uses the data it has collected for free ions, this includes information for the entire periodic table.
* From the CRC Handbook of Chemistry and Physics it uses information on the ionization energies.
* From the Materials Project it takes information on known stable crystal structures.
* From Morrison's book, it uses information for the crystal field parameters for a limited range () of hosts.

In this notebook data from this sources is parsed and collected in a standard useable format.
For each of this a detailed scheme for each is presented and some simple view of the datasets are created.

Furthermore, in this notebook we also define and refine the classes that qdef. These classes include: 

* atom: a class for a building block of the universe,
* crystal: a class for a crystal as parsed from cif files,
* sub: a class for a substitutional ion in a specific host.

## Databases

### The basics

#### parsing dictionaries for names, symbols, and atomic number
saving them to pickles

In [2]:
nsa = pd.read_csv('/Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/elementsymbols.csv',
                  names=['name','symbol','atomic number'])
name_to_symb = [(n,s) for n,s in zip(nsa['name'],nsa['symbol'])]
name_to_symb = dict(name_to_symb)
name_to_num = [(n,nu) for n,nu in zip(nsa['name'],nsa['atomic number'])]
name_to_num = dict(name_to_num)

symb_to_name = [(s,n) for s,n in zip(nsa['symbol'],nsa['name'])]
symb_to_name = dict(symb_to_name)
symb_to_num = [(s,nu) for s,nu in zip(nsa['symbol'],nsa['atomic number'])]
symb_to_num = dict(symb_to_num)

num_to_name = [(n,s) for n,s in zip(nsa['atomic number'],nsa['name'])]
num_to_name = dict(num_to_name)
num_to_symb = [(s,nu) for s,nu in zip(nsa['atomic number'],nsa['symbol'])]
num_to_symb = dict(num_to_symb)

pickles = ['name_to_symb', 'name_to_num',
         'symb_to_name', 'symb_to_num',
         'num_to_name', 'num_to_symb']
collection = [name_to_symb, name_to_num, symb_to_name, symb_to_num,
             num_to_name, num_to_symb]
for pkl_name, thing in zip(pickles, collection):
    with open('./data/%s.pkl' % pkl_name, 'wb') as f:
        pickle.dump(thing,f)

In [728]:
# this just to load it
name_to_symb = pickle.load(open('./data/name_to_symb.pkl','rb'))
name_to_num  = pickle.load(open('./data/name_to_num.pkl','rb'))
symb_to_name = pickle.load(open('./data/symb_to_name.pkl','rb'))
symb_to_num  = pickle.load(open('./data/symb_to_num.pkl','rb'))
num_to_name  = pickle.load(open('./data/num_to_name.pkl','rb'))
num_to_symb  = pickle.load(open('./data/num_to_symb.pkl','rb'))
atom_symbs   = list(symb_to_name.keys())
atom_names   = list(name_to_num.keys())

In [729]:
ionization_data = pickle.load(open('/Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/ionization_data.pkl','rb'))['data']

## Parsing Atomic data from Mathematica

In [742]:
wolf = open('/Users/juan/Temp/misthios.csv','r').read().split('\n')
cub = {}
bits = [("Block","s",None), # string
        ("CAS number","s",None),
        ("Crystal Structure","s",None),
        ("Electrical Type","s",None),
        ("Electron Config","li",None), # list of ints
        ("Electron Config String","lat",None),
        ("Electronegativity","f",1),
        ("Period","i",None),
        ("Mass Magnetic Susceptibility","cf","m^3/kg"),
        ("Memberships","ls",None),
        ("Series","s",None),
        ("Van der Waals radius","cf","pm"),
        ("Atomic Mass","cf","amu"),
        ("Atomic Radius","cf","pm"),
        ("Covalent Radius","cf","pm"),
        ("Neutron Cross Section","cf","bn"),
        ("Universe Abundance","cf",1),
        ("Group","s",None)
       ]
def tofloat(s):
    if s == '':
        return np.nan
    else:
        return float(s)
for bit, aline in zip(bits, wolf):
    line = aline.replace('{',"").replace('}',"").replace('"','').split(',')
    if bit[1] == 's': 
        clean_line = [l.strip() for l in line]
    elif bit[1] == 'f': # a float
        clean_line = [tofloat(l.strip()) for l in line]
    elif bit[1] == 'li':
        clean_line = list(map(lambda x: eval(('[['+x).strip().replace(']],',']]')),(','.join(line)).split('[[')[1:]))
    elif bit[1] == 'cf': # CForm
        clean_line = [tofloat(l.strip()) for l in aline.replace('List(','').replace('"','').replace(')','').split(',')]
    elif bit[1] == 'i': # integers
        clean_line = [int(l.strip()) for l in line]
    elif bit[1] == 'ls': # list of sting lists
        chunks = aline.replace('""','"').replace('","',',')[2:-2].replace("{","[").replace("}",']').split('],[')
        clean_line = list(map(lambda x: eval('[%s]' % x), chunks))
    elif bit[1] == 'lat':
        clean_line = aline.replace('"','').split(',')
    cub[bit[0]] = {i:c for i,c in zip(range(1,119),clean_line)}

In [743]:
for at_num, econf in cub['Electron Config'].items():
    ecg = {}
    for principal, orbs in enumerate(econf):
        principal = principal+1
        for magnum, lorb in zip(['s','p','d','f'],orbs):
#             print(str(principal)+magnum+"^%d"%lorb)
            ecg[str(principal)+magnum] = lorb
    cub['Electron Config'][at_num] = ecg

In [746]:
pickle.dump(cub,
    open('/Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/atomicgoodies.pkl','wb'))

### Free the ions!

## Classes

In [747]:
# load data
name_to_symb = pickle.load(open('./data/name_to_symb.pkl','rb'))
name_to_num  = pickle.load(open('./data/name_to_num.pkl','rb'))
symb_to_name = pickle.load(open('./data/symb_to_name.pkl','rb'))
symb_to_num  = pickle.load(open('./data/symb_to_num.pkl','rb'))
num_to_name  = pickle.load(open('./data/num_to_name.pkl','rb'))
num_to_symb  = pickle.load(open('./data/num_to_symb.pkl','rb'))
atom_symbs   = list(symb_to_name.keys())
atom_names   = list(name_to_num.keys())

nistdf = pd.read_pickle('./data/nist_atomic_spectra_database_levels.pkl')
spinData = pd.read_pickle('./data/spindata.pkl')
atomicGoodies = pickle.load(open('./data/atomicGoodies.pkl','rb'))

### Atom

In [748]:
class Atom():
    '''
    From these lots of things come.
    '''
    def __init__(self, kernel):
        '''
        Object can be initialized either by giving an atomic number,
        element name, or element symbol.
        '''
        if kernel in range(1, 119):
            self.atomic_number = kernel
            self.symbol = num_to_symb[kernel]
            self.name = num_to_name[kernel]
        elif (kernel in atom_names) or (kernel.lower() in atom_names):
            self.name = kernel.lower()
            self.symbol = name_to_symb[self.name]
            self.atomic_number = name_to_num[self.name]
        elif kernel in atom_symbs:
            self.symbol = kernel
            self.atomic_number = symb_to_num[kernel]
            self.name = symb_to_name[kernel]
        else:
            raise ValueError('to initialize input must be either an atomic' +
                             ' number, element name, or element symbol')
        # a dictionary with known ionization energies at different stages
        if self.symbol in ['Db','Sg','Bh','Hs','Mt','Ds','Rg',
                           'Cn','Nh','Fl','Mc','Lv','Ts','Og']:
            self.ionization_energies = []
        else:
            self.ionization_energies = ionization_data[self.symbol]
        # a dataframe with level data as compiled by NIST
        self.nist_data = nistdf[nistdf['Element'] == self.symbol]
        # a dataframe with isotopic data
        self.isotope_data = spinData[spinData['atomic_number'] == self.atomic_number]
        # additional data
        # the electronegativity of the free neutral atom
        self.electronegativity = atomicGoodies["Electronegativity"][self.atomic_number]
        # a latex string for the electron configuration of the ground state
        self.ground_state_config = atomicGoodies["Electron Config String"][self.atomic_number]
        # crystal structure of its most common solid form
        self.common_crystal_structure = atomicGoodies["Crystal Structure"][self.atomic_number]
        # electron configuration
        self.electron_configuration = atomicGoodies["Electron Config String"][self.atomic_number]
        # Van der Waals radius
        self.van_der_waals_radius =  atomicGoodies["Van der Waals radius"][self.atomic_number]
        # Atomic radius
        self.atomic_radius = atomicGoodies["Atomic Radius"][self.atomic_number]
        # Covalent radius
        self.covalent_radius = atomicGoodies["Covalent Radius"][self.atomic_number]
        # Block
        self.block = atomicGoodies["Block"][self.atomic_number]
        # Period
        self.period = atomicGoodies["Period"][self.atomic_number]
        # Series
        self.series = atomicGoodies["Series"][self.atomic_number]
        # Group
        self.group = atomicGoodies["Group"][self.atomic_number]
        
    def level_diagram(self, charge, min_energy=-np.inf, max_energy=np.inf):
        '''make a nice plot of the levels of the ion with the given charge'''
        cmap = plt.cm.RdYlGn
        datum = self.nist_data[an_atom.nist_data['Charge'] == charge]
        energy_levels = datum['Level (eV)']
        configs = datum['Configuration']
        if charge == 0:
            fig_name = '%s' % (self.symbol)
            latex_name = fig_name
        else:
            fig_name = '%s +%d' % (self.symbol, charge)
            latex_name = '{%s}^{+%d}' % (self.symbol, charge)
        plt.close(fig_name)
        fig, ax = plt.subplots(figsize=(2, 5), num=fig_name)
        annot = ax.annotate("", xy=(0, 0), xytext=(-20, 20),
                            textcoords="offset points",
                            bbox=dict(boxstyle="round", fc="w"),
                            arrowprops=dict(arrowstyle="->"))
        annot.set_visible(False)

        for level_energy in energy_levels:
            if ((level_energy < max_energy) & (level_energy > min_energy)):
                ax.plot([0, 1], [level_energy]*2, 'k-', lw=0.5)
        level_anchors = ax.scatter([0]*len(datum['Level (eV)']),
                                   datum['Level (eV)'],
                                   c='k', s=4)
        ax.axes.xaxis.set_visible(False)
        plt.ylabel('E / eV')
        ax.set_title('$%s$' % latex_name)
        plt.tight_layout()

        def update_annot(ind):
            pos = level_anchors.get_offsets()[ind["ind"][0]]
            annot.xy = pos
            indices = ind["ind"]
            text = (configs[indices[0]] + '\n' +
                    ('%.3f eV' % (energy_levels[indices[0]])))
            annot.set_text(text)
            annot.get_bbox_patch().set_facecolor('white')

        def hover(event):
            vis = annot.get_visible()
            if event.inaxes == ax:
                cont, ind = level_anchors.contains(event)
                if cont:
                    update_annot(ind)
                    annot.set_visible(True)
                    fig.canvas.draw_idle()
                else:
                    if vis:
                        annot.set_visible(False)
                        fig.canvas.draw_idle()

        fig.canvas.mpl_connect("motion_notify_event", hover)
        plt.show()

    def __repr__(self):
        return '%s : %s : %d' % (self.name, self.symbol, self.atomic_number)

    def __str__(self):
        return '%s : %s : %d' % (self.name, self.symbol, self.atomic_number)


In [718]:
atoms = {i:Atom(i) for i in range(1,119)}

In [727]:
atoms[6].nist_data[atoms[6].nist_data['Level (eV)'] == 0]

,Element,Charge,Configuration,Term,J,g,Level (eV),Uncertainty (eV),Splitting,Leading percentages,Reference,Lande,J_uncertain,J_has_issues
0,C,0,2s2.2p2,3P,[0],1,0.0,1.600000e-07,,98 ...,L20057,,False,False
0,C,1,2s2.2p,2P*,[1/2],2,0.0,NaN,,NaN,L7288,NaN,False,False
0,C,2,1s2.2s2,1S,[0],1,0.0,NaN,,NaN,,NaN,False,False
0,C,3,1s2.2s,2S,[1/2],2,0.0,NaN,,NaN,L7288,NaN,False,False
0,C,4,1s2,1S,[0],1,0.0,NaN,,NaN,L11091,NaN,False,False
0,C,5,1s,2S,[1/2],2,0.0,NaN,,NaN,L3620,NaN,False,False


In [556]:
class PeriodicTable():
    def __init__(self):
        self.atoms = {i:Atom(i) for i in range(1,119)}

In [557]:
periodicTable = PeriodicTable()

In [692]:
import math

In [717]:
plt.close('ptable')
fig, ax = plt.subplots(num='ptable',figsize=(9.5,5.5))
margin = 0.02
annotations = {atom.atomic_number :  '%.0f pm ' % atom.atomic_radius for atom in periodicTable.atoms.values() if not(math.isnan(atom.atomic_radius))}
font_multiplier = 1.5
for atom in periodicTable.atoms.values():
    if atom.group != 'f-block':
        group_loc = int(atom.group)
        row = -atom.period
    else:
        if (57 <= atom.atomic_number <= 70):
            group_loc = atom.atomic_number - 57+3
            row = -9+0.5
        else:
            group_loc = atom.atomic_number - 89+3
            row = -10+0.5
    if atom.atomic_number in annotations.keys():
        plt.text(group_loc+0.5, row-0.8, annotations[atom.atomic_number], ha='center', va='center', fontsize=4*font_multiplier)
    plt.text(group_loc+0.5, row-0.375, atom.symbol,ha='center',va='top',fontsize=7*font_multiplier, weight='bold')
    plt.text(group_loc+0.5, row-margin-0.1, atom.atomic_number,ha='center',va='top',fontsize=4*font_multiplier)
    plt.plot([group_loc+margin, group_loc+1-margin, group_loc+1-margin, group_loc+margin, group_loc+margin],
            [ row-margin, row-margin, row-1+margin, row-1+margin, row-margin],'k-',lw=0.5)
ax.set_xlim(1,19)
ax.set_ylim(-11,-1)
ax.axis('off')
ax.set_aspect('equal')
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [716]:
plt.figure()
radii = np.array([[atom.atomic_number, atom.atomic_radius] for atom in periodicTable.atoms.values()])
plt.plot(radii[:,0], radii[:,1])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [715]:
radii

array([False])

In [540]:
periodicTable = {i: Atom(i) for i in range(1,119)}

In [542]:
display(Math(periodicTable[12].electron_configuration))

<IPython.core.display.Math object>

In [516]:
from IPython.display import display, Math

In [520]:
display(Math(r'\text[Ne]3\text{s}^2'))

<IPython.core.display.Math object>

In [515]:
display(r'$\frac{x}{y}$')

'$\\frac{x}{y}$'